<table style="width: 100%; border-style: none;">
<tr style="border-style: none">
<td style="border-style: none; width: 1%; font-size: 16px">Institut f&uuml;r Theoretische Physik<br /> Universit&auml;t zu K&ouml;ln</td>
<td style="border-style: none; width: 1%; font-size: 16px">&nbsp;</td>
<td style="border-style: none; width: 1%; text-align: right; font-size: 16px">Prof. Dr. Simon Trebst<br />Peter Br&ouml;cker</td>
</tr>
</table>
<hr>
<h1 style="font-weight:bold; text-align: center; margin: 0px; padding:0px;">Computerphysik</h1>
<h1 style="font-weight:bold; text-align: center; margin: 0px; padding:0px;">Vorlesung 4 - Verarbeitung Experimenteller Daten</h1>
<hr>
<h3 style="font-weight:bold; text-align: center; margin: 0px; padding:0px; margin-bottom: 20px;">Sommersemester 2016</h3>

**Website:** [http://www.thp.uni-koeln.de/trebst/Lectures/2016-CompPhys.shtml](http://www.thp.uni-koeln.de/trebst/Lectures/2016-CompPhys.shtml)

## Beschreibung des Experiments

Im Rahmen dieser Vorlesung wollen wir erarbeiten, wie man experimentelle Daten, die zum Beispiel während eines Praktikumsversuchs gewonnen wurden, mit Julia verarbeiten kann. Ein im Grundstudium häufig durchgeführtes Experiment ist das [Pohlsche Rad](https://de.wikipedia.org/wiki/Pohlsches_Rad) als Beispiel für resonante und chaotische Schwingvorgänge. Daran angelehnt betrachten wir die folgende Differentialgleichung, die einen durch eine Kraft $F$ getriebenen Oszillator mit Reibungskoeffizienten $b$ beschreibt.

$\quad m\dfrac{d^2x}{dt^2} + b\dfrac{dx}{dt}+kx = F \sin(\omega t)$

$\quad \dfrac{d^2x}{dt^2} + \dfrac{b}{m} \dfrac{dx}{dt}+\dfrac{k}{m}x = \dfrac{1}{m}F \sin(\omega t)$

 Das Ziel des Versuchs ist es zunächst die Federkonstante $k$ zu bestimmen und dann durch einen Fit den Reibungskoeffizienten $b$ zu finden.

## 1. Installation von Paketen

In [ ]:
Pkg.add("DataFrames")
Pkg.add("ExcelReaders")
run(`easy_install --user uncertainties`)
run(`easy_install --user xlrd`)
Pkg.add("LsqFit")

## 2. Download der experimentellen Daten

In [ ]:
download("https://docs.google.com/spreadsheets/d/1p3zgKZ_lFKdiefQUCR-4UNMieWcINNXGUypvJzrA9jo/pub?output=xlsx", "experimental_data.xlsx");
download("https://docs.google.com/spreadsheets/d/1p3zgKZ_lFKdiefQUCR-4UNMieWcINNXGUypvJzrA9jo/pub?output=csv", "experimental_data.csv");

## 3. Datenformate

**CSV** Format - **c**omma **s**eparated **v**alues

Universell lesbar und leicht automatisch generierbar. Wie sehen sie aus?

In [ ]:
f = open("experimental_data.csv", "r")
for ln in eachline(f)
    print(ln)
end

Wir können die Daten mit dem Paket *DataFrames* einlesen (und schreiben):

In [ ]:
using DataFrames

period_data_csv = readtable("experimental_data.csv", separator=',') # 1. zeile ist die Ueberschrift
#period_data_csv = readtable("experimental_data.csv", separator=',', header=false) # ohne Ueberschrift

In [ ]:
# : bedeutet: alle Zeilen der Matrix
period_data_csv[:, 1]

Alternativen: Excel, OpenDocument (LibreOffice, OpenOffice, ...)
- Gut geeignet für übersichtliche Eingabe per Hand und Vorschauplots
- Mehrere Blätter (=Versuche) pro Dokument möglich
- Paket *ExcelReaders* ermöglicht Zugriff

In [ ]:
using ExcelReaders

# readxl(filename, "Sheetname!cell range")
period_data_xls = readxl("experimental_data.xlsx", "Freies Pendel!A2:D11")
resonance_data_xls = readxl("experimental_data.xlsx", "Resonanzmessung!A3:C85")
;

## 4. Darstellung der Ergebnisse im $\LaTeX$ - Look

Beide Einlesemethoden, CSV und Excel, liefern den Typ *DataArray* zurück, den wir für die Darstellung noch einmal in einen gewöhnlichen Array konvertieren müssen:

In [ ]:
resonance_x_data = convert(Array{Float64, 1}, resonance_data_xls[:, 1])
resonance_y_data = convert(Array{Float64, 1}, resonance_data_xls[:, 2])
resonance_y_weigths = convert(Array{Float64, 1}, resonance_data_xls[:, 3])
;

Insbesondere in den fortgeschritteneren Praktika werden Sie umfangreiche Protokolle anfertigen, zumeist in $\LaTeX$. Um das Aussehen der Grafiken dem des Textes anzupassen, können Sie das folgende Codeschnipsel für Ihre Plots verwenden:

In [ ]:
# plottet hier - theoretisch -.-
#...
# fmt="." stellt das Aussehen der Datenpunkte ein

## 5. Bestimmung von $k$

In der Praxis ist das Ausrechnung von Fehlern kombinierter Größen oft eine komplizierte Angelegenheit. Bei der Eingabe langer Ausdrücke für die partiellen Ableitungen in ein Tabellenkalkulationsprogramm schleichen sich oft Fehler ein, die schwierig zu finden sind. Deshalb zeigen wir Ihnen wie man dieses Problem zumindest teilweise an das Paket *uncertainties* übergeben kann.

In [ ]:
using PyCall

@pyimport uncertainties.unumpy as unp # lade python-pakete
@pyimport uncertainties.umath as umath

Wir erstellen einen Array aus den gemessenen Daten für die Masse und die Schwingungsperiode

In [ ]:
mass_data = unp.uarray(period_data_xls[:,1], period_data_xls[:,2]) # y-werte, y-fehler
period_data = unp.uarray(period_data_xls[:,3], period_data_xls[:,4]) # y-werte, y-fehler

Für jeden Array können wir uns die Messwerte ...

In [ ]:
unp.nominal_values(mass_data)

... und deren Fehler ausgeben lassen:

In [ ]:
unp.std_devs(mass_data)

Die Berechnung von Mittelwerten ist noch nicht implementiert, deswegen werden wir es an dieser Stelle tun:

In [ ]:
function uarray_avg(array)
    array_size = size(array)[1]
    array_sum  = umath.fsum(array) # float-sum
    return pyeval("a_sum / a_size", a_sum=array_sum, a_size=array_size) # lasse Python rechnen
end

Zu guter Letzt müssen wir noch den Ausdruck für unsere kombinierte Größe eingeben, und zwar in einer Form, die ein wenig an ein Tabellenkalkulationprogramm erinnert:

$T = 2\pi\sqrt{\frac{k}{m}}$

In [ ]:
mass_avg = uarray_avg(mass_data)
period_avg = uarray_avg(period_data)

pyeval("4 * pi * pi * m / (T * T)", T=period_avg, m=mass_avg, pi=pi)

## 6. Bestimmung von $b$ durch fitten

Aus der exakten Lösung wissen wir, dass die Amplitude die folgende Form besitzt:

$\quad A = \dfrac{F}{\sqrt{m^2 (\frac{k}{m} - \omega^2)^2 + b^2 \omega^2}}$

Bis auf den Parameter $b$ (Reibungsparameter) kennen wir alle Größen, so dass wir ein Modell erstellen können, dass wir an die Daten fitten:

In [ ]:
using LsqFit

In [ ]:
x_data = convert(Array{Float64, 1}, resonance_data_xls[:, 1])
y_data = convert(Array{Float64, 1}, resonance_data_xls[:, 2])
y_weights = convert(Array{Float64, 1}, resonance_data_xls[:, 3])
;

In [ ]:
using PyPlot

F = 1.65
m = 1.34
k = 0.96

model(x, p) = F ./ sqrt(m^2 * (k/m - x .*x).^2 + p[1]^2 .* x.^2) +p[2] # p: fit-parameter, p[1] = b; p[2] fuer syst. Fehler

fit = curve_fit(model, x_data, y_data, 1./y_weights, [0.5, 0.5])
println(fit.param)
sleep(0.1) # print at once
errorbar(x_data, y_data, yerr=y_weights, fmt=".")
plot(x_data, model(x_data, fit.param))
;

Zur Bestimmung des Parameters gehört auch die Angabe eines Fehlers, den wir aus der Kovarianzmatrix erhalten:

In [ ]:
cov_mat = sqrt(estimate_covar(fit))
println("Δb = ", cov_mat[1,1])